# Calibrating the model using Spotpy

In [1]:
from pathlib import Path
import datetime
import pandas as pd
from numpy import sqrt, mean
import spotpy
import plotly.graph_objects as go
from hydrogr import InputDataHandler, ModelGr4j

## Load the calibration data

Datasets are available as Pandas dataframe pickle in the project repository.

In [2]:
data_path = Path.cwd().parent / 'data'
df = pd.read_pickle(data_path / 'L0123001.pkl')
df.columns = ['date', 'precipitation', 'temperature', 'evapotranspiration', 'flow', 'flow_mm']
df.index = df['date']
print(df.head())

                 date  precipitation  temperature  evapotranspiration  \
date                                                                    
1984-01-01 1984-01-01            4.1          0.5                 0.2   
1984-01-02 1984-01-02           15.9          0.2                 0.2   
1984-01-03 1984-01-03            0.8          0.9                 0.3   
1984-01-04 1984-01-04            0.0          0.5                 0.3   
1984-01-05 1984-01-05            0.0         -1.6                 0.1   

               flow  flow_mm  
date                          
1984-01-01   2640.0   0.6336  
1984-01-02   3440.0   0.8256  
1984-01-03  12200.0   2.9280  
1984-01-04   7600.0   1.8240  
1984-01-05   6250.0   1.5000  


## Spotpy interface

Tu use Spotpy, we need to create an interface for our model. In particular, this interface provides an objective function methods to evaluate the model results.
Here we use a simple RMSE between computed and observed flow.

In [3]:
class SpotpySetup(object):
    """
    Interface to use the model with spotpy
    """

    def __init__(self, data):
        self.data = data
        self.model_inputs = InputDataHandler(ModelGr4j, self.data)
        self.params = [spotpy.parameter.Uniform('x1', 0.01, 300.0),
                       spotpy.parameter.Uniform('x2', 0.0, 5.0),
                       spotpy.parameter.Uniform('x3', 0.01, 300.0),
                       spotpy.parameter.Uniform('x4', 0.5, 10.0),
                       ]

    def parameters(self):
        return spotpy.parameter.generate(self.params)

    def simulation(self, vector):
        simulations = self._run(x1=vector[0], x2=vector[1], x3=vector[2], x4=vector[3])
        return simulations

    def evaluation(self):
        return self.data['flow_mm'].values

    def objectivefunction(self, simulation, evaluation):
        res = - sqrt(mean((simulation - evaluation) ** 2.0))
        return res
    
    def _run(self, x1, x2, x3, x4):
        parameters = {"X1": x1, "X2": x2, "X3": x3, "X4": x4}
        model = ModelGr4j(parameters)
        outputs = model.run(self.model_inputs.data)
        return outputs['flow'].values

## Calibration

To calibrate our model we select a sub-period from our dataset.
To find optimal model parameters, we can test several optimisation algorithm available in spotpy.

In [4]:
# Reduce the dataset to a sub period :
start_date = datetime.datetime(1998, 1, 1, 0, 0)
end_date = datetime.datetime(2008, 1, 1, 0, 0)
mask = (df['date'] >= start_date) & (df['date'] <= end_date)
calibration_data = df.loc[mask]

spotpy_setup = SpotpySetup(calibration_data)

# sampler = spotpy.algorithms.mc(spotpy_setup, dbformat='ram')
sampler = spotpy.algorithms.mcmc(spotpy_setup, dbformat='ram')
# sampler = spotpy.algorithms.mle(spotpy_setup, dbformat='ram')
# sampler = spotpy.algorithms.lhs(spotpy_setup, dbformat='ram')
# sampler = spotpy.algorithms.sceua(spotpy_setup, dbformat='ram')
# sampler = spotpy.algorithms.demcz(spotpy_setup, dbformat='ram')
# sampler = spotpy.algorithms.sa(spotpy_setup, dbformat='ram')
# sampler = spotpy.algorithms.rope(spotpy_setup, dbformat='ram')

sampler.sample(2000)
results=sampler.getdata() 
best_parameters = spotpy.analyser.get_best_parameterset(results)
print(best_parameters)

Initializing the  Markov Chain Monte Carlo (MCMC) sampler  with  2000  repetitions
The objective function will be maximized
Starting the MCMC algotrithm with 2000 repetitions...
Initialize  1  chain(s)...
Initialize database...
['csv', 'hdf5', 'ram', 'sql', 'custom', 'noData']
Beginn of Random Walk
231 of 2000, maximal objective function=-0.689079, time remaining: 00:00:15
Acceptance rates [%] =94.81
489 of 2000, maximal objective function=-0.689079, time remaining: 00:00:12
Acceptance rates [%] =92.86
767 of 2000, maximal objective function=-0.689079, time remaining: 00:00:10
Acceptance rates [%] =91.41
1057 of 2000, maximal objective function=-0.646482, time remaining: 00:00:07
Acceptance rates [%] =91.3
1398 of 2000, maximal objective function=-0.646482, time remaining: 00:00:04
Acceptance rates [%] =90.86
1767 of 2000, maximal objective function=-0.646482, time remaining: 00:00:02
Acceptance rates [%] =90.79

*** Final SPOTPY summary ***
Total Duration: 13.88 seconds
Total Repetiti

## Validation

Finally we validate our calibration on another time period

In [6]:
parameters = list(best_parameters[0])
parameters = {"X1": parameters[0], "X2": parameters[1], "X3": parameters[2], "X4": parameters[3]}

start_date = datetime.datetime(1989, 1, 1, 0, 0)
end_date = datetime.datetime(1999, 12, 31, 0, 0)
mask = (df['date'] >= start_date) & (df['date'] <= end_date)
validation_data = df.loc[mask]

model = ModelGr4j(parameters)
outputs = model.run(validation_data)

# Remove the first year used to warm up the model :
filtered_input = validation_data[validation_data.index >= datetime.datetime(1990, 1, 1, 0, 0)]
filtered_output = outputs[outputs.index >= datetime.datetime(1990, 1, 1, 0, 0)]

rmse = sqrt(mean((filtered_output['flow'] - filtered_input['flow_mm'].values) ** 2.0))
print(rmse)


0.8031153380049457


In [7]:
fig = go.Figure([
    go.Scatter(x=filtered_output.index, y=filtered_output['flow'], name="Calculated"),
    go.Scatter(x=filtered_input.index, y=filtered_input['flow_mm'], name="Observed"),
])
fig.show()